<a href="https://colab.research.google.com/github/leticia-chen/DeepLearning_DeblurGan2_SR/blob/main/DeblurGanv2_SR_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# 請輸入資料夾之所在位置
FOLDERNAME = 'Colab\ Notebooks/DeblurGanv2-SR'
assert FOLDERNAME is not None, "[!] Enter the foldername."

In [ ]:
# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/MyDrive/{}'.format(FOLDERNAME))

In [ ]:
# Get to the folder we are at
%cd drive/MyDrive/$FOLDERNAME/

/content/drive/MyDrive/Colab Notebooks/DeblurGanv2-SR


In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('using device:', device)

using device: cpu


In [ ]:
%ls

Generator: Residual block, Upsample block, Generator net

In [ ]:
import math
import torch
from torch import nn

class ResidualBlock(nn.Module):
  def __init__(self, channels):                                 
    super(ResidualBlock, self).__init__()
    # in_channel X out_channel X kernel X padding
    # channels = 64                                                   # 这个数字待定?
    self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
    self.bn1 = nn.BatchNorm2d(channels, 0.8)                          # 此架构的作者并没放 0.8, 其它架构作者皆有放 0.8, 0.8 意义为何 ?
    self.prelu = nn.PReLU(channels)
    self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
    self.bn2 = nn.BatchNorm2d(channels, 0.8)

  def forward(self, x):
    short_cut = x
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.prelu(x)
    x = self.conv2(x)
    x = self.bn2(x)

    return x + short_cut

In [ ]:
class UpsampleBlock(nn.Module):
  def __init__(self, in_channels, up_scale):                           # 参数待定
    super(UpsampleBlock, self).__init__()
    self.conv = nn.Conv2d(in_channels, in_channels*up_scale**2, kernel_size=3, padding=1)      # 64->256
    self.pixel_suffle = nn.PixelShuffle(up_scale)
    self.prelu = nn.PReLU(in_channels)

  def forward(self, x):
    x = self.conv(x)
    x = self.pixel_suffle(x)
    x = self.prelu(x)
    return(x)


In [ ]:
class NetG(nn.Module):
  def __init__(self, num_residual=16):                       # 参数待定      

    super(NetG, self).__init__()

    # First layer
    self.conv1 = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=9, padding=4),                 # out 64 是否要改
        nn.PReLU(64)
    )

    # Residual blocks
    self.res_blocks = []
    for _ in range(num_residual):
      self.res_blocks.append(ResidualBlock(64))
    self.res_blocks = nn.Sequential(*self.res_blocks)

    # Second conv layer pos residual
    self.conv2 = nn.Sequential(
        nn.Conv2d(64, 64, kernel_size=3, padding=1),
        nn.BatchNorm2d(64, 0.8)
    )

    # Upsampling layer
    self.upsample = []
    for _ in range(2):
      self.upsample.append(UpsampleBlock(64, 2))
    self.upsample = nn.Sequential(*self.upsample)
    # 原写法
    # self.upsample = [UpsampleBlock(64, 2) for _ in range(2)]
    # self.upsample.append(nn.Conv2d(64,3), kernel_size=9, padding=4)
    # self.upsample = nn.Sequential(*self.upsample)

    # the last conv layer
    self.conv3 = nn.Sequential(
        nn.Conv2d(64, 3, kernel_size=9, stride= 1, padding=4),
        nn.Tanh())

  def forward(self, x):
    x = self.conv1(x)
    short_cut = x
    x = self.res_blocks(x)
    x = self.conv2(x)
    x = x + short_cut
    x = self.upsample(x)
    out = self.conv3(x)

    return out


Inicialization

In [ ]:
netG = NetG().to(device)

Demo

In [ ]:
# from torchvision.io import read_image
# import cv2
import numpy as np
import torchvision.transforms as transforms
from PIL import Image
from torch.autograd import Variable
# import sys
# import itertools
from torchvision.utils import save_image, make_grid
# import torch.nn.functional as F
import torch
# from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

# Better performance mode-checkpoint
model08 = 'models_checkpoint_by_Jay/generator_8.pth'
model41 = 'models_checkpoint_by_Jay/generator_41.pth'
model56 = 'models_checkpoint_by_Jay/generator_56.pth'
model47 = 'models_checkpoint_by_Jay/generator_47.pth'

models = [model08, model41, model56, model47]

def trans(img):

  image = Image.open(img).convert('RGB')
  w, h = image.size
  # print('img:', image.size) # (1280, 720)

  transform = transforms.Compose([
      transforms.Resize((h // 4, w // 4), Image.BICUBIC), 
      transforms.ToTensor(), 
      transforms.Normalize(mean, std)])
  
  image = transform(image)
  # print(image.shape)                         # torch.Size([3, 720, 1280])
  # print(next(iter(image)).shape)          # torch.Size([720, 1280])
  image = image.unsqueeze(dim=0)      # 如果没有这行，回传的 tensor 的 channel 不见
  # print(image.shape)                        # torch.Size([1, 3, 180, 320])

  return image


In [ ]:
def get_img(img):

  model_name = []
  model_num = []
  fake_img_lst = []
  for model in models:

    num = ''
    for n in model:
      if n.isdigit():
        num += n
    model_num.append(num)

    # map_location=torch.device() 当为 cpu 需有 map_location
    netG.load_state_dict(torch.load(model, map_location=torch.device(device))) 
    netG.eval()

    img = Variable(img.to(device))
    img = img.data    # Variable 张量转为 tensor
    netG.zero_grad()                           
    fake_imgs = netG(img)

    model_name.append(model)
    fake_imgs = nn.functional.interpolate(fake_imgs, scale_factor=4)
    fake_imgs = make_grid(fake_imgs, padding=3, normalize=True)
    fake_imgs = fake_imgs.permute(1,2,0)            # 没有会出错
    fake_img_lst.append(fake_imgs) 
  
  # print(model_name)
  # print(model_num)
  plt.figure(figsize=(17,10))
  for j in range(len(fake_img_lst)):
    out_img = fake_img_lst[j].squeeze()
    # out_img = out_img.permute(1,2,0)
    ax = plt.subplot(2,2,j+1)
    ax.set_title('Model'+ str(model_num[j]))
    ax.imshow(out_img)
  plt.show()


In [ ]:
import os
# import numpy as np
# from torch.utils.data import DataLoader

demo_files = 'imgs_for_demo'

demo_file_paths = []
files = os.listdir(demo_files)                                

for file in files:
  demo_file = os.path.join(demo_files, file)
  demo_file_paths.append(demo_file)

for img in demo_file_paths:
  img = trans(img)
  img = get_img(img)

print(demo_file_paths)
print(len(demo_file_paths))
# for k in range(9, len(demo_file_paths)):
#   img = trans(demo_file_paths[k])
#   img = get_img(img)
